# Context


* When an applicant submits a loan it may be either Accepted or Rejected.

, there are 3 possible loan statuses that may occur:
    1. Currently active - loan is still being paid off and it is before the due date
    2. Fully Paid - The loan was fully paid in the allotted time
    3. Default - The applicant was unable to pay the loan fully before the due date and the loan is considered "defaulted"

# Data Collection

In this notebook, we will create the dataset by pulling from LendersClub data and then take the necessary measures to **Clean the Data** and apply imputation techniques 

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import kagglehub # For the data
import os 

### Load the dataset

In [14]:
# Download latest version of kagglehub
path = kagglehub.dataset_download("wordsforthewise/lending-club")
os.chdir(r"C:\Users\henry\.cache\kagglehub\datasets\wordsforthewise\lending-club\versions\3") # Change into path containing the data


accepted_df = pd.read_csv("accepted_2007_to_2018Q4.csv.gz")
rejected_df = pd.read_csv("rejected_2007_to_2018Q4.csv.gz")

C:\Users\henry\AppData\Local\Temp\ipykernel_9588\707967294.py:6: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv("accepted_2007_to_2018Q4.csv.gz")


### EDA

In [32]:
# Drop Cols (axis=1) if all values = NaN (how='all)

accepted_df = accepted_df.dropna(axis=1, how='all')
rejected_df = rejected_df.dropna(axis=1, how='all')

In [33]:
print("Shape------------------------------------- ")
print(accepted_df.shape)
print(rejected_df.shape)

print("Info-------------------------------------- ")
print(accepted_df.info())
print(rejected_df.info())

print("Dtypes------------------------------------ ")
print(accepted_df.dtypes)
print(rejected_df.dtypes)

Shape------------------------------------- 
(2260701, 150)
(27648741, 9)
Info-------------------------------------- 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 150 entries, id to settlement_term
dtypes: float64(112), object(38)
memory usage: 2.5+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648741 entries, 0 to 27648740
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Loan Title            object 
 3   Risk_Score            float64
 4   Debt-To-Income Ratio  object 
 5   Zip Code              object 
 6   State                 object 
 7   Employment Length     object 
 8   Policy Code           float64
dtypes: float64(3), object(6)
memory usage: 1.9+ GB
None
Dtypes------------------------------------ 
id                        object
loan_amnt                float64
funded_amnt              float64


In [40]:
print(list(accepted_df.columns))

['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_am

In [36]:
accepted_df.tail()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
2260696,88985880,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,Vice President,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260697,88224441,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,Program Manager,...,NaN,NaN,Cash,Y,Mar-2019,ACTIVE,Mar-2019,10000.0,44.82,1.0
2260698,88215728,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,Customer Service Technician,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260699,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2260700,Total amount funded in policy code 2: 521953170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# TODO: Clean the data - impute, check missing vals - etc. 
# TODO: Make target variable - Default loan or not? 
# 